In [3]:
#Made following tutorial series by Patrick Loeber with some edits 
import torch
import numpy as np

In [5]:
###--- 1) Basics of Tensors ---###
#Tensor is basically an array with some dimension and shape, but unlike np arrays they can be moved to GPUs for ML
# and can be used for gradient tracking
#Uninitialized tensor with given shape, faster than initializing with 0 or 1
x = torch.empty(2, 3)
print(x)


x = torch.zeros(2, 3)
print(x)
x = torch.ones(2, 3)
print(x)

#we can specify data type of elements, float32 is the default for nn params
x = torch.rand(2, 3, dtype = torch.float32)
print(x)

tensor([[1.2640e-27, 4.5856e-41, 1.2639e-27],
        [4.5856e-41, 9.4075e-28, 4.5856e-41]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.0051, 0.3095, 0.6836],
        [0.3987, 0.1796, 0.5950]])


In [ ]:
#Tensor Operations
x, y = torch.rand(2,2)

#elementwise ops
print(x, y, x+y, x-y, x*y, x/y)

#slicing
z = torch.rand(6,3, 4)
print(z[:, 0]) # = z[:, 0, :]
print(z[:, 0, 2])
print(z[0, 2, 0].item())

In [ ]:
#Reshaping
x = torch.rand(4, 4)

# Changes the shape to whatever is specified
#-1 allows pytorch to determine the correct size to maintain same number of elements
print(x.view(-1))

print(x.view(8, -1))

#reshape does the same thing but either returns a view or copy of the data
print(x.reshape(8, -1))

In [ ]:
#Casting between array and tensor
x = torch.rand(3)
y = np.array(x) #or x.numpy()
z = torch.Tensor(y)

print(x, y, z)

In [18]:
#Moving between cpu and gpu
x = torch.rand(3)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda available")
    x = x.to("cuda")
    print(x.device)
else:
    print("cuda unavailable; using cpu")


x =x.to("cpu")
print(x.device)

x = x.to(device)
print(x.device)

cuda available
cuda:0
cpu
cuda:0


In [ ]:
###--- 2) Autograd ---###
#from torchviz import make_dot
# keeps track of computational graph for us when we create tensors, streamlines backprop to calculate gradients
# which will be used in gradient update step using optim.step()

x = torch.rand(3, requires_grad = True)
w = torch.rand(2, 3, requires_grad = True)
b = torch.rand(2, requires_grad = True)
y = w@x + b
z = y.mean()

#dot = make_dot(z, params={'y': y, 'x': x})
#dot.format = 'png'
#dot.render('tensor_ops_graph')

z.backward()

print(w.grad)
print(b.grad)

In [7]:
import torchvision
import torchvision.transforms as transforms

In [57]:
batch_size = 128

train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor(),
                                         download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 39.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.19MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.7MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.24MB/s]


In [55]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [54]:
print(next(iter(train_loader))[1])

tensor([7, 8, 4, 6, 1, 3, 0, 0, 7, 8, 1, 9, 6, 8, 9, 4, 5, 6, 2, 0, 1, 9, 3, 1,
        3, 5, 3, 2, 4, 9, 9, 9, 3, 7, 8, 4, 9, 4, 5, 1, 3, 2, 4, 2, 4, 3, 9, 9,
        1, 3, 9, 5, 6, 7, 0, 3, 1, 8, 1, 4, 6, 4, 3, 7, 8, 0, 4, 6, 4, 2, 8, 9,
        2, 7, 2, 1, 9, 7, 4, 6, 7, 5, 8, 8, 6, 4, 5, 1, 1, 9, 0, 6, 1, 1, 6, 5,
        4, 9, 0, 9, 6, 9, 9, 7, 4, 5, 8, 9, 4, 2, 0, 6, 5, 5, 2, 1, 6, 4, 0, 9,
        9, 7, 1, 9, 9, 3, 4, 9])


In [12]:
num_epochs = 5
num_classes = 10
learning_rate = 0.001

In [16]:
###--- x) Training a CNN-based model on MNIST ---###
import torch.nn as nn

#Define the model

class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [22]:
model = CNNModel(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    
    for batch_num, (images, labels)  in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
         
        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_num+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, batch_num+1, total_step, loss.item()))

#Training loop
#Learn early stopping, regularization

Epoch [1/5], Step [100/469], Loss: 0.1784
Epoch [1/5], Step [200/469], Loss: 0.0950
Epoch [1/5], Step [300/469], Loss: 0.0593
Epoch [1/5], Step [400/469], Loss: 0.0150
Epoch [2/5], Step [100/469], Loss: 0.0601
Epoch [2/5], Step [200/469], Loss: 0.0182
Epoch [2/5], Step [300/469], Loss: 0.0248
Epoch [2/5], Step [400/469], Loss: 0.0419
Epoch [3/5], Step [100/469], Loss: 0.0234
Epoch [3/5], Step [200/469], Loss: 0.0507
Epoch [3/5], Step [300/469], Loss: 0.0243
Epoch [3/5], Step [400/469], Loss: 0.0661
Epoch [4/5], Step [100/469], Loss: 0.0160
Epoch [4/5], Step [200/469], Loss: 0.0068
Epoch [4/5], Step [300/469], Loss: 0.0216
Epoch [4/5], Step [400/469], Loss: 0.0123
Epoch [5/5], Step [100/469], Loss: 0.0333
Epoch [5/5], Step [200/469], Loss: 0.0466
Epoch [5/5], Step [300/469], Loss: 0.0329
Epoch [5/5], Step [400/469], Loss: 0.0050


In [23]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 97.98 %
